In [10]:
import pandas as pd
import numpy as np

In [34]:
df = pd.DataFrame({"close": [45, 23, 65, 34, 45, 23, 45, 66], "raw_peaks": [0,1,0,0,1,0,0,0]})
df

,close,raw_peaks
0,45,0
1,23,1
2,65,0
3,34,0
4,45,1
5,23,0
6,45,0
7,66,0


In [53]:
def nearby_extremes(df, threshold: float, max_sequence: int, forward_pass: bool, reverse_pass: bool):
    """Identify values beside peaks/valleys that are within a
    threshold distance and re-label them"""

    df.rename(columns={df.columns[1]: "raw_peaks"}, inplace=True)
    df_holder = []
    df_holder.append(df["raw_peaks"])

    # Forward Pass
    if forward_pass:
        forward_df = (pd.concat(
            [
                df,
                df.mask(df["raw_peaks"] == 0).fillna(
                    method="ffill").rename(columns=lambda n: "prev_" + n),
            ],
            axis=1
        )
                      .eval("within_threshold = abs(close-prev_close)/prev_close < @threshold")
                      .eval("mask = within_threshold and raw_peaks == 0")
                      .eval("is_new_extreme = within_threshold.mask(mask).fillna(method='ffill')")
                      .eval("forward_extremes = prev_raw_peaks.where(is_new_extreme).fillna(0)")
                      ["forward_extremes"]
                      .astype(int))
        forward_df["counter"] = 1
        forward_df["sequence"] = forward_df.groupby("forward_extremes")["counter"].cumsum()
        forward_df["sequence_mask"] = forward_df["sequence"] < max_sequence
        forward_df["forward_extremes_cut"] = forward_df["forward_extremes"].where(sequence_mask)
        df_holder.append(forward_df)

    # Reverse Pass
    if reverse_pass:
        df = df.sort_index(ascending=False)
        reverse_df = (pd.concat(
            [
                df,
                df.mask(df["raw_peaks"] == 0).fillna(
                    method="ffill").rename(columns=lambda n: "prev_" + n),
            ],
            axis=1
        )
                      .eval("within_threshold = abs(close-prev_close)/prev_close < @threshold")
                      .eval("mask = within_threshold and raw_peaks == 0")
                      .eval("is_new_extreme = within_threshold.mask(mask).fillna(method='ffill')")
                      .eval("reverse_extremes = prev_raw_peaks.where(is_new_extreme).fillna(0)")
                      ["reverse_extremes"]
                      .astype(int)).sort_index()
        df_holder.append(reverse_df)

    # Merging
    merged_df = pd.concat(df_holder, axis=1)
    final_df = merged_df.sum(axis=1).clip(lower=-1, upper=1)

    return final_df

In [54]:
nearby_extremes(df, 0.01, 4, True, True)

UndefinedVariableError: name 'df' is not defined

In [21]:
d.groupby("value")["i"].cumsum()

0     1
1     2
2     1
3     1
4     2
5     2
6     3
7     4
8     1
9     5
10    3
Name: i, dtype: int64

In [23]:
d.groupby("value").cumsum()

,i
0,1
1,2
2,1
3,1
4,2
5,2
6,3
7,4
8,1
9,5


In [31]:
pd.eval("d.groupby('value').cumsum()")

,i
0,1
1,2
2,1
3,1
4,2
5,2
6,3
7,4
8,1
9,5


In [52]:
d["value"].groupby(d["value"])

In [66]:
import numpy as np
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
X, y = make_multilabel_classification(n_classes=10, n_labels=3, random_state=0)
clf = MultiOutputClassifier(LogisticRegression()).fit(X, y)
clf.predict(X)

array([[0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 1, 0, 1, 0, 1],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 1, 0, 1, 0, 0],
       [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 1, 0, 1],
       [0, 0, 1, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
       [0, 1, 1, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 1, 0, 1],
       [0, 1, 1, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 1, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0,

In [62]:
X.shape

(100, 20)